# Emotet
- Probably Emotet
- MD5 8baa9b809b591a11af423824f4d9726a
- [VirusTotal](https://www.virustotal.com/gui/file/6393fe8dd4721190f240e22feeb769675b6194a70cabd5a415c2364686a9089c/detection)
- [Malware Link](https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-114-3/)

## Imports and DF

In [10]:
import pandas as pd

In [11]:
emotet = pd.read_csv(r"../../data/raw/csv/emotet.csv")

## Labelling

In [14]:
ip_list = [
    "103.228.200.37", "103.245.153.70", "119.59.124.163", "13.107.39.203", "13.107.42.23",
    "162.144.46.185", "178.23.244.51", "188.126.72.179", "192.163.239.60", "192.168.0.1",
    "192.168.0.18", "192.168.0.65", "192.229.211.108", "195.219.57.34", "20.22.113.133",
    "20.99.132.105", "20.99.133.109", "20.99.184.37", "20.99.185.48", "20.99.186.246",
    "200.159.128.132", "202.44.54.4", "23.202.154.36", "23.209.116.25", "23.216.147.64",
    "23.216.147.76", "23.56.238.72", "40.125.120.53", "64.207.134.54", "96.16.108.70"
]

# Update the "Label" column where "Destination IP" or "Source IP" is in the list
emotet.loc[
    (emotet['Destination IP'].isin(ip_list)) | (emotet['Source IP'].isin(ip_list)),
    'Label'
] = 'Emotet'

In [17]:
emotet["Label"].value_counts()

Label
Benign    44252
Emotet    36745
Name: count, dtype: int64

## Flag Formatting

In [18]:
# List of TCP flags
tcp_flag_mapping = [
    'FIN',
    'SYN',
    'RST',
    'PSH',
    'ACK',
    'URG',
    'ECE',
    'CWR',
    'NS',
]

# Function to format the flags
def format_flags(flags_str):
    # Capitalize the flags string first
    flags_str = flags_str.upper()

    # Initialize an empty list to store found flags
    formatted_flags = []

    # Check each individual flag from the mapping
    for flag in tcp_flag_mapping:
        if flag in flags_str:
            formatted_flags.append(flag)

    # Join the found flags with commas
    return ','.join(formatted_flags)

# Apply the format_flags function to the "flags" column
emotet['Flags'] = emotet['Flags'].apply(format_flags)

In [21]:
emotet

,Duration,Source IP,Destination IP,Source Port,Destination Port,Protocol,Flags,Packets,Bytes,Flows,Label
0,0.093964,10.0.2.102,224.0.0.252,52334,5355,LLMNR,,2,128,1,Benign
1,1.501686,10.0.2.102,10.0.2.255,137,137,NBNS,,3,276,1,Benign
2,0.000000,10.0.2.102,8.8.8.8,59869,53,DNS,,1,76,1,Benign
3,0.000000,8.8.8.8,10.0.2.102,53,59869,DNS,,1,181,1,Benign
4,0.000000,10.0.2.102,8.8.8.8,64935,53,DNS,,1,76,1,Benign
...,...,...,...,...,...,...,...,...,...,...,...
80992,9.001526,10.0.2.102,103.1.186.61,54564,8080,TCP,SYN,3,194,1,Benign
80993,16.209324,10.0.2.102,202.44.54.4,54565,8080,TCP,"SYN,ACK",4,228,1,Emotet
80994,15.978455,202.44.54.4,10.0.2.102,8080,54565,TCP,"FIN,SYN,ACK",3,166,1,Emotet
80995,0.000000,10.0.2.102,202.44.54.4,54565,8080,HTTP,"PSH,ACK",1,474,1,Emotet


## Saving to CSV

In [22]:
emotet.to_csv('../../data/labelled/emotet_data.csv', index=False)